# Optimize Initial Conditions
## Yabox

In [1]:
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [2]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=220*GB,
         lru_evict=True,
         driver_object_store_memory=500*MB,num_gpus=5,num_cpus=1,
         ignore_reinit_error=True) # , include_webui=False)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-10-22 10:28:57,091	INFO resource_spec.py:212 -- Starting Ray with 219.97 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-22 10:28:57,409	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

# Load New and Process Data from website data.brasil.io

In [3]:
%reload_ext autoreload
%autoreload 2
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

(pid=2768) ray.get_gpu_ids(): 4
(pid=2768) CUDA_VISIBLE_DEVICES: 4


# Functions to Load Processed Data

In [4]:
def load_confirmed(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Confirmed-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

def load_recovered(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Recovered-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

def load_dead(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Deaths-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

# Load solver

In [5]:
%reload_ext autoreload
%autoreload 2
import LearnerICRayNoLoadBH_v3 as L 

# Data for Countries

In [6]:
dfparam = pd.read_csv("data/param.csv")
countries=dfparam.country
popEst = pd.read_csv("data/WPP2019_TotalPopulationBySex.csv")
popEst['popTotal']=pd.to_numeric(popEst.PopTotal, errors='coerce')

for country in countries:
    if country=="US":
        country2="United States of America"    
    else:
        country2=country
    dfparam.loc[dfparam.country==country,'popTotal']=popEst.loc[popEst.Location==country2].loc[popEst.Time==2020].iloc[0,8]*1000
    dfparam.loc[dfparam.country==country,'s0']=popEst.loc[popEst.Location==country2].loc[popEst.Time==2020].iloc[0,8]*1000
    
display(dfparam)
    

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH,popTotal
0,Brazil,3/2/20,200,2.125594e+08,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,2.125594e+08
1,China,1/28/20,200,1.439324e+09,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,1.439324e+09
2,Italy,2/28/20,200,6.046183e+07,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,6.046183e+07
3,US,2/20/20,200,3.310026e+08,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,3.310026e+08
4,India,3/10/20,200,1.380004e+09,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,1.380004e+09


# Functions for Optimization

In [7]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(country,e0,a0,date, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(country,date)
        recovered = load_recovered(country,date)
        data = load_confirmed(country,date)-recovered-dead
        cleanRecovered=False
        s0, deltaDate, i0, d0, r0, startNCases, weigthCases, weigthRecov, weightDeath = point
        end_date=datetime.strptime(date, "%m/%d/%y") + timedelta(days=deltaDate)
        f=L.Learner.remote(country, end_date.strftime("%m/%d/%y"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, weigthCases, weigthRecov, weightDeath, \
                           cleanRecovered, version, data, dead, recovered, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [8]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdeath,date,startNCases, 
        predict_range, version):

    bounds=[(20e3,s0),(-2,2),(0,500), (0,500),(0,500),(0,500),\
              (0.15,1.0),(0.1,0.5),(0.1,0.5)]
    maxiterations=500
    f=create_f(country,e0,a0,date, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead,best_params
    
    return p

# Main Code

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [10]:
countries=dfparam.country
display(countries)
allCountries=True
version="010"
gc.enable()

optimal=[]
if allCountries:
    for country in countries:
        #remove previous history file
        strFile='./results/history_'+country+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec,wdth, pop = parameters
        optimal.append(opt.remote(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdth,date,startNCases, 
                                            predict_range, version))        
else:
    country = "Brazil" 
    #remove previous history file
    strFile='./results/history_'+country+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec,wdth, pop = parameters
    optimal.append(opt.remote(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdth,date,startNCases, 
                                        predict_range, version))            

0    Brazil
1     China
2     Italy
3        US
4     India
Name: country, dtype: object

In [ ]:
optimal=ray.get(optimal)


2020-10-22 10:29:05,320	WARNING worker.py:1090 -- WARNING: 3 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-22 10:29:06,292	WARNING worker.py:1090 -- WARNING: 4 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-22 10:29:06,689	WARNING worker.py:1090 -- WARNING: 5 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-22 10:29:07,686	WARNING worker.py:1090 -- WARNING: 6 PYTHON workers have been started. This could be a resu

(pid=2853) warning: basinhopping: local minimization failure
(pid=2853) basinhopping step 0: f 1.9741e+09
(pid=2912) basinhopping step 0: f 2.26905e+11
(pid=2886) warning: basinhopping: local minimization failure
(pid=2886) basinhopping step 0: f 3.00784e+09
(pid=2827) basinhopping step 0: f 1.09482e+12
(pid=2853) basinhopping step 1: f 1.9741e+09 trial_f 2.14377e+16 accepted 0  lowest_f 1.9741e+09
(pid=2938) warning: basinhopping: local minimization failure
(pid=2938) basinhopping step 0: f 6.12663e+11
(pid=2938) basinhopping step 1: f 6.12663e+11 trial_f 1.40827e+15 accepted 0  lowest_f 6.12663e+11
(pid=2827) basinhopping step 1: f 1.09482e+12 trial_f 4.28778e+14 accepted 0  lowest_f 1.09482e+12
(pid=2886) basinhopping step 1: f 3.00784e+09 trial_f 3.99961e+11 accepted 0  lowest_f 3.00784e+09
(pid=2938) basinhopping step 2: f 6.12663e+11 trial_f 9.70373e+11 accepted 0  lowest_f 6.12663e+11
(pid=2912) basinhopping step 1: f 2.26905e+11 trial_f 4.57859e+14 accepted 0  lowest_f 2.26905e

2020-10-22 10:31:13,720	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=2886) basinhopping step 8: f 2.64355e+09 trial_f 2.69862e+09 accepted 0  lowest_f 2.64355e+09
(pid=2827) basinhopping step 9: f 8.05757e+11 trial_f 6.1503e+14 accepted 0  lowest_f 8.05757e+11
(pid=2938) basinhopping step 10: f 1.90157e+11 trial_f 8.89365e+15 accepted 0  lowest_f 1.90157e+11
(pid=2938) basinhopping step 0: f 1.81075e+11
(pid=3288) basinhopping step 0: f 1.08543e+11
(pid=2938) basinhopping step 1: f 1.75544e+11 trial_f 1.75544e+11 accepted 1  lowest_f 1.75544e+11
(pid=2938) found new global minimum on step 1 with function value 1.75544e+11
(pid=2938) basinhopping step 2: f 1.75544e+11 trial_f 1.9606e+11 accepted 0  lowest_f 1.75544e+11
(pid=2853) warning: basinhopping: local minimization failure
(pid=2853) basinhopping step 10: f 1.9741e+09 trial_f 2.1654e+09 accepted 0  lowest_f 1.9741e+09
(pid=2853) basinhopping step 0: f 2.13938e+09


2020-10-22 10:31:22,002	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

(pid=30502) basinhopping step 1: f 3.21116e+11 trial_f 8.75733e+11 accepted 0  lowest_f 3.21116e+11
(pid=30404) basinhopping step 10: f 2.15887e+09 trial_f 4.61509e+12 accepted 0  lowest_f 2.15887e+09
(pid=30545) warning: basinhopping: local minimization failure
(pid=30545) basinhopping step 0: f 1.35804e+09
(pid=30404) basinhopping step 0: f 2.1574e+09
(pid=30404) basinhopping step 1: f 2.1574e+09 trial_f 2.16075e+09 accepted 0  lowest_f 2.1574e+09
(pid=30502) warning: basinhopping: local minimization failure
(pid=30502) basinhopping step 2: f 3.21116e+11 trial_f 2.51773e+14 accepted 0  lowest_f 3.21116e+11
(pid=30404) basinhopping step 2: f 2.15624e+09 trial_f 2.15624e+09 accepted 1  lowest_f 2.15624e+09
(pid=30404) found new global minimum on step 2 with function value 2.15624e+09


2020-10-22 15:28:08,045	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=30460) basinhopping step 6: f 1.12184e+11 trial_f 2.76111e+11 accepted 0  lowest_f 1.12184e+11
(pid=30254) warning: basinhopping: local minimization failure
(pid=30254) basinhopping step 9: f 4.4455e+11 trial_f 5.95268e+11 accepted 0  lowest_f 4.4455e+11
(pid=30545) basinhopping step 1: f 1.35804e+09 trial_f 7.20608e+12 accepted 0  lowest_f 1.35804e+09
(pid=30559) basinhopping step 0: f 2.99137e+09
(pid=30559) basinhopping step 1: f 2.99137e+09 trial_f 3.29087e+12 accepted 0  lowest_f 2.99137e+09
(pid=30460) warning: basinhopping: local minimization failure
(pid=30460) basinhopping step 7: f 1.12184e+11 trial_f 2.7571e+11 accepted 0  lowest_f 1.12184e+11
(pid=30502) basinhopping step 3: f 3.21116e+11 trial_f 2.14965e+14 accepted 0  lowest_f 3.21116e+11
(pid=30254) warning: basinhopping: local minimization failure
(pid=30254) basinhopping step 10: f 4.37587e+11 trial_f 4.37587e+11 accepted 1  lowest_f 4.37587e+11
(pid=30254) found new global minimum on step 10 with function value 4

2020-10-22 15:28:33,403	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=30545) basinhopping step 3: f 1.35804e+09 trial_f 1.35848e+09 accepted 0  lowest_f 1.35804e+09
(pid=30589) basinhopping step 0: f 1.35363e+14
(pid=30589) basinhopping step 1: f 1.35363e+14 trial_f 1.72064e+15 accepted 0  lowest_f 1.35363e+14
(pid=30545) basinhopping step 4: f 1.31741e+09 trial_f 1.31741e+09 accepted 1  lowest_f 1.31741e+09
(pid=30545) found new global minimum on step 4 with function value 1.31741e+09
(pid=30502) basinhopping step 4: f 3.21116e+11 trial_f 2.1542e+14 accepted 0  lowest_f 3.21116e+11
(pid=30559) basinhopping step 3: f 2.99137e+09 trial_f 1.98805e+12 accepted 0  lowest_f 2.99137e+09
(pid=30460) basinhopping step 8: f 1.12184e+11 trial_f 1.29765e+14 accepted 0  lowest_f 1.12184e+11
(pid=30589) warning: basinhopping: local minimization failure
(pid=30589) basinhopping step 2: f 9.18923e+11 trial_f 9.18923e+11 accepted 1  lowest_f 9.18923e+11
(pid=30589) found new global minimum on step 2 with function value 9.18923e+11
(pid=30559) warning: basinhopping:

In [ ]:
for i in range(0,len(countries)):    

    country=countries[i]
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%m/%d/%y")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%m/%d/%y")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    